In [ ]:
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Dropout, MaxPooling2D, Conv2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
train_data = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train_path = "../input/cassava-leaf-disease-classification/train_images"
test_path = "../input/cassava-leaf-disease-classification/test_images"
train_data.head()

In [ ]:
train_data.info()

In [ ]:
img = plt.imread(train_path+"/"+train_data["image_id"][0])
print(img.shape)
plt.imshow(img)

In [ ]:
file = open("../input/cassava-leaf-disease-classification/label_num_to_disease_map.json")
label_dict = json.load(file)
label_dict

In [ ]:
train_data.groupby('label').count().plot(kind='bar', title='Target class distribution', figsize=(15,7), grid=1)

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(15, 7))
for i, img in enumerate(train_data.groupby('label').first().reset_index().values):
    ax[i].imshow(plt.imread(train_path + f"/{img[1]}"))
    ax[i].set_title(img[0])
    ax[i].axis('off')
fig.suptitle('Image Samples', fontsize=18); 

In [ ]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 64
INPUT_SHAPE = (224, 224, 3)
CLASSES = 5

image_datagen = ImageDataGenerator(
    rescale=1/255.0,
    validation_split = 0.3 
)

train_data.label = train_data.label.astype('str')

In [ ]:
train_generator = image_datagen.flow_from_dataframe(
    train_data,
    directory=train_path,
    x_col="image_id",
    y_col="label",
    target_size=IMAGE_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    subset="training",
    shuffle=True,
    seed=42,
    class_mode="sparse"
)

In [ ]:
test_generator = image_datagen.flow_from_dataframe(
    train_data,
    directory=train_path,
    x_col="image_id",
    y_col="label",
    target_size=IMAGE_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    subset="validation",
    shuffle=True,
    seed=42,
    class_mode="sparse"
)

In [ ]:
def getModel():
    model = Sequential()

    model.add(Conv2D(16, (5,5), input_shape=INPUT_SHAPE, activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2)))
    
    model.add(Conv2D(32, (5,5), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(64, (5,5), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2)))
    
    model.add(Conv2D(128, (3,3), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(256, (3,3), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2)))


    model.add(Flatten())

    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.3))

    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))

    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.3))

    model.add(Dense(CLASSES, activation="softmax"))
    
    return model

In [ ]:
temp = getModel()
temp.summary()

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss', 
    min_delta=0,
    patience=10, 
    verbose=1, 
    restore_best_weights=True)

In [ ]:
SGD_model = getModel()
SGD_model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.SGD(), metrics=["accuracy"])

In [ ]:
train_generator.n

In [ ]:
%%time
SGD_history = SGD_model.fit(train_generator,
                validation_data = test_generator,
                steps_per_epoch = train_generator.n // BATCH_SIZE,
                validation_steps = test_generator.n // BATCH_SIZE,
                epochs=10,
                callbacks=[early_stopping])

In [ ]:
RMS_model = getModel()
RMS_model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.RMSprop(), metrics=["accuracy"])

In [ ]:
%%time
RMS_history = RMS_model.fit(train_generator,
                validation_data = test_generator,
                steps_per_epoch = train_generator.n // BATCH_SIZE,
                validation_steps = test_generator.n // BATCH_SIZE,
                epochs=10,
                callbacks=[early_stopping])

In [ ]:
Adam_model = getModel()
Adam_model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

In [ ]:
%%time
Adam_history = Adam_model.fit(train_generator,
                validation_data = test_generator,
                steps_per_epoch = train_generator.n // BATCH_SIZE,
                validation_steps = test_generator.n // BATCH_SIZE,
                epochs=10,
                callbacks=[early_stopping])

In [ ]:
plt.plot(Adam_history.history['loss'])
plt.plot(Adam_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
train_generator_small = image_datagen.flow_from_dataframe(
    train_data[:1000],
    directory=train_path,
    x_col="image_id",
    y_col="label",
    target_size=IMAGE_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    subset="training",
    shuffle=True,
    seed=42,
    class_mode="sparse"
)
test_generator_small = image_datagen.flow_from_dataframe(
    train_data[:1000],
    directory=train_path,
    x_col="image_id",
    y_col="label",
    target_size=IMAGE_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    subset="validation",
    shuffle=True,
    seed=42,
    class_mode="sparse"
)

In [ ]:
lr_rate = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))

In [ ]:
LRS_model = getModel()
LRS_model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-8), metrics=["accuracy"])

In [ ]:
%%time
LRS_history = LRS_model.fit(train_generator_small,
                validation_data = test_generator_small,
                steps_per_epoch = train_generator_small.n // BATCH_SIZE,
                validation_steps = test_generator_small.n // BATCH_SIZE,
                epochs=100,
                callbacks=[lr_rate])

In [ ]:
plt.semilogx(LRS_history.history["lr"], LRS_history.history["loss"])

In [ ]:
LR_model = getModel()
LR_model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"])

In [ ]:
%%time
LR_history = LR_model.fit(train_generator,
                validation_data = test_generator,
                steps_per_epoch = train_generator.n // BATCH_SIZE,
                validation_steps = test_generator.n // BATCH_SIZE,
                epochs=10)

In [ ]:
plt.plot(LR_history.history['loss'])
plt.plot(LR_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
image_datagen_aug = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=5,
    zoom_range=0.1,
    shear_range=0.05,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator_aug = image_datagen_aug.flow_from_dataframe(
    train_data,
    directory=train_path,
    x_col="image_id",
    y_col="label",
    target_size=IMAGE_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    subset="training",
    shuffle=True,
    seed=42,
    class_mode="sparse"
)

test_generator_aug = image_datagen_aug.flow_from_dataframe(
    train_data,
    directory=train_path,
    x_col="image_id",
    y_col="label",
    target_size=IMAGE_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    subset="validation",
    shuffle=True,
    seed=42,
    class_mode="sparse"
)

In [ ]:
temp = next(train_generator)
print(temp[0].shape)
plt.imshow(temp[0][0])
plt.show()

In [ ]:
AUG_model = getModel()
AUG_model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"])

In [ ]:
%%time
AUG_history = AUG_model.fit(train_generator_aug,
                validation_data = test_generator_aug,
                steps_per_epoch = train_generator_aug.n // BATCH_SIZE,
                validation_steps = test_generator_aug.n // BATCH_SIZE,
                epochs=10)

In [ ]:
from tensorflow.keras.applications import ResNet50V2

In [ ]:
base_model = ResNet50V2(include_top=True, input_shape=INPUT_SHAPE)
base_model.layers[-2]

In [ ]:

newLayer = Dense(CLASSES, activation='softmax')
outLayer = newLayer(base_model.layers[-2].output)

ResNet_model = Model(inputs=base_model.input, outputs=outLayer)

for layer in base_model.layers[:-1]:
    layer.trainable = False
    
ResNet_model.summary()

In [ ]:
ResNet_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
%%time
ResNet_history = ResNet_model.fit(
    train_generator,
    validation_data = test_generator,
    epochs = 10
)

In [ ]:
ALL_accuracy = [SGD_history.history['accuracy'][-1],
                RMS_history.history['accuracy'][-1],
                Adam_history.history['accuracy'][-1],
                LR_history.history['accuracy'][-1],
                AUG_history.history['accuracy'][-1], 
                ResNet_history.history['accuracy'][-1]]

ALL_val_accuracy = [SGD_history.history['val_accuracy'][-1],
                RMS_history.history['val_accuracy'][-1],
                Adam_history.history['val_accuracy'][-1],
                LR_history.history['val_accuracy'][-1],
                AUG_history.history['val_accuracy'][-1],
                ResNet_history.history['val_accuracy'][-1]]

ALL_loss = [SGD_history.history['loss'][-1],
                RMS_history.history['loss'][-1],
                Adam_history.history['loss'][-1],
                LR_history.history['loss'][-1],
                AUG_history.history['loss'][-1],
                ResNet_history.history['loss'][-1]]

ALL_val_loss = [SGD_history.history['val_loss'][-1],
                RMS_history.history['val_loss'][-1],
                Adam_history.history['val_loss'][-1],
                LR_history.history['val_loss'][-1],
                AUG_history.history['val_loss'][-1],
                ResNet_history.history['val_loss'][-1]]

In [ ]:
experiments = {"experiment": ["Custom model + SGD", "Custom model + RMS", "Custom model + Adam", "Custom model + Adam + LearningRate", "Custom model + Adam + LR + Image augment", "ResNet + Adam"],
              "train_accuracy": ALL_accuracy,
              "test_accuracy": ALL_val_accuracy}

In [ ]:
table = pd.DataFrame(experiments)
table